In [2]:
from IPython.core.display import display, HTML
display(HTML("<style> .container { width:80% !important;}</style>"))
display(HTML("<style> div#header { width:100%; }</style>"))
display(HTML('<style>pre { white-space: pre !important; }</style>'))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

/tmp/ipykernel_1100071/2922149680.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
os.environ['SPARK_MAJOR_VERSION'] = '3'
os.environ['SPARK_HOME'] = '/usr/sdp/current/spark3-client/'
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'python'
os.environ['PYSPARK_PYTHON'] = '/opt/sdp/stockfish/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/sdp/stockfish/bin/python'
os.environ['LD_LIBRARY_PATH'] = '/opt/python/virtualenv/jupyter/lib'
sys.path.insert(0, '/usr/sdp/current/spark3-client/python/')
sys.path.insert(0, '/usr/sdp/current/spark3-client/python/lib/py4j-0.10.9.3-src.zip')
import pyspark.sql.functions as F

from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, asc, desc, length, udf, round
from pyspark.sql.types import ArrayType, StringType, FloatType, IntegerType

In [4]:
conf = SparkConf().setAppName(app_name)\
    .setMaster("yarn")\
    .set('spark.dynamicAllocation.enabled', 'True')\
    .set('spark.executor.memory','48g')\
    .set('spark.driver.memory','48g')\
    .set('spark.executor.memoryOverhead','36g')\
    .set('spark.driver.maxResultSize','512g')\
    .set('spark.executor.cores', '15')\
    .set('spark.driver.maxResultSize','512g')\
    .set('spark.dynamicAllocation.initialExecutors', 1)\
    .set('spark.dynamicAllocation.minExecutors', 1)\
    .set('spark.dynamicAllocation.maxExecutors', 10)\
    .set('spark.port.maxRetries', '150')\
    .set('spark.driver.cores', '1')\
    .set('spark.driver.memoryOverhead', '128g')\
    .set('spark.shuffle.service.enabled', 'true')\
    .set('spark.sql.parquet.datetimeRebaseModeInRead', 'CORRECTED')\
    .set('spark.sql.parquet.int96RebaseModeInWrite', 'CORRECTED')\
    .set('spark.sql.parquet.int96RebaseModeInRead', 'CORRECTED')\
    .set('spark.sql.parquet.enableVectorizedReader', 'false')\
    .set('spark.kryoserializer.buffer.maxValue', '2044018')\
    .set('spark.sql.autoBroadcastJoinThreshold', -1)\
    .set('spark.sql.broadcastTimeout', -1)\
    .set("spark.sql.catalogImplementation", "hive")\
    .set("mapreduce.input.fileinputformat.input.dir.recursive", 'true')\
    .set('spark.hive.mapred.supports.subdirectories', 'true')\
    .set('spark.sql.hive.convertMetastoreParquet', 'true')

In [5]:
ss = SparkSession \
    .builder \
    .appName(app_name) \
    .config(conf=conf) \
    .getOrCreate()
ss

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/09 19:14:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/09 19:14:17 WARN HiveConf: HiveConf of name hive.mapred.supports.subdirectories does not exist
25/01/09 19:14:17 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
25/01/09 19:14:18 WARN Client: Exception encountered while connecting to the server 
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyException): Operation category READ is not supported in state standby. Visit https://s.apache.org/sbnn-error
	at org.apache.hadoop.security.SaslRpcClient.saslConnect(SaslRpcClient.java:376)
	at org.apache.hadoop.ipc.Client$Connection.setupSaslConnection(Client.java:623)
	at org.apache.hadoop.ipc.Client$Connection.access$2300(Client.java:414)
	at org.ap

In [6]:
sc = ss.sparkContext
sc.setLogLevel('ERROR')

In [7]:
df = ss.read.parquet('hdfs://arnsdpldbr2/user/team/team_uim_crm/18281950/chess_merged/all_data_18_22')

In [8]:
df.show(20,truncate=False)

+-----------+----------+-----+--------------------+---------------+-------+----------+--------------------------------------------------------+---+-----------------------------------------------------------------------------+--------+--------+-----------+-----------------------------------------+---------+----------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
df.count()

5927957

In [10]:
df = df.dropDuplicates()

In [11]:
df.columns

['Link',
 'Date',
 'Round',
 'White',
 'Black',
 'Result',
 'Tournament',
 'CurrentPosition',
 'ECO',
 'ECOUrl',
 'WhiteElo',
 'BlackElo',
 'TimeControl',
 'Termination',
 'StartTime',
 'EndDate',
 'EndTime',
 'Moves',
 'Moves_length',
 'Tournament_long',
 'Moves_clean',
 'sf_n1_d22',
 'sf_n10_d18']

In [12]:
df_part = df.sample(False, 0.005,0)

In [13]:
df_part.count()

29548

### Перевод нотации

In [14]:
import chess

def change_notation(moves):

    board = chess.Board()
    stockfish_moves = []

    for move in moves:
        try:
            board.push_san(move)
            last_move_uci = board.peek().uci()
            stockfish_moves.append(last_move_uci)
        except:
            return None
    return stockfish_moves

In [15]:
change_notation_udf = udf(change_notation, ArrayType(StringType()))
df_part = df_part.withColumn('Moves_sf', change_notation_udf(df['Moves_clean']))

In [16]:
df_part.show(truncate=False)

+-----------+----------+-----+-------------------+------------------+-------+--------------+--------------------------------------------------------+---+-----------------------------------------------------------------------------------------+--------+--------+-----------+-----------------------------------------+---------+----------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Проверим, что была небольшая доля убранных игр

In [17]:
df_part = df_part.filter( 'Moves_sf is not null')

### Разделение на 3 части 

In [18]:
def moves_separation(mov_len):
    deb_start = 0
    mit_start = min(10, mov_len)
    part = (mov_len - 10)//5
    mittel = 3*part
    end_start = mit_start + mittel
    if mit_start % 2:
        mit_start = mit_start - 1
    if end_start % 2:
        end_start = end_start - 1
    return mit_start, end_start, mov_len

In [19]:
moves_separation_udf = udf(moves_separation, ArrayType(StringType()))
df_part = df_part.withColumn('parts_idxs', moves_separation_udf(df_part['Moves_length']))

In [20]:
df_part.show(10)

+-----------+----------+-----+-----------------+---------------+-------+--------------+--------------------+---+--------------------+--------+--------+-----------+------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|       Link|      Date|Round|            White|          Black| Result|    Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|       Termination|StartTime|   EndDate| EndTime|               Moves|Moves_length|     Tournament_long|         Moves_clean|           sf_n1_d22|          sf_n10_d18|            Moves_sf|   parts_idxs|
+-----------+----------+-----+-----------------+---------------+-------+--------------+--------------------+---+--------------------+--------+--------+-----------+------------------+---------+----------+--------+--------------------+------------+--------------------+-

### Подсчет доли совпадений со stockfish на 1 линии для белых и черных в каждой партии

In [21]:
def coincidences_1(moves, sf, parts):
    real = sf 
    pred = moves
    
    coincidences_black_m = 0
    coincidences_white_m = 0
    
    coincidences_black_e = 0
    coincidences_white_e = 0
    
    coincidences_black_d = 0
    coincidences_white_d = 0

    for i in range(len(real)):
        if pred[i] == real[i][0]['_1']:
            if i%2 == 0:
                if float(parts[0]) > i:
                    coincidences_white_d += 1
                elif float(parts[1]) > i:
                    coincidences_white_m += 1
                else:
                    coincidences_white_e += 1
            else:
                if float(parts[0]) > i:
                    coincidences_black_d += 1
                elif float(parts[1]) > i:
                    coincidences_black_m += 1
                else:
                    coincidences_black_e += 1
                
    return  [coincidences_white_d, coincidences_white_m, coincidences_white_e], [coincidences_white_d, coincidences_white_m, coincidences_black_e]

In [22]:
coincidences_1_udf = udf(coincidences_1, ArrayType(ArrayType(StringType())))
df_part = df_part.withColumn('coin_w_d', coincidences_1_udf(df_part['Moves_sf'], df_part['sf_n1_d22'], df_part['parts_idxs'])[0][0])
df_part = df_part.withColumn('coin_w_m', coincidences_1_udf(df_part['Moves_sf'], df_part['sf_n1_d22'], df_part['parts_idxs'])[0][1])
df_part = df_part.withColumn('coin_w_e', coincidences_1_udf(df_part['Moves_sf'], df_part['sf_n1_d22'], df_part['parts_idxs'])[0][2])

df_part = df_part.withColumn('coin_b_d', coincidences_1_udf(df_part['Moves_sf'], df_part['sf_n1_d22'], df_part['parts_idxs'])[1][0])
df_part = df_part.withColumn('coin_b_m', coincidences_1_udf(df_part['Moves_sf'], df_part['sf_n1_d22'], df_part['parts_idxs'])[1][1])
df_part = df_part.withColumn('coin_b_e', coincidences_1_udf(df_part['Moves_sf'], df_part['sf_n1_d22'], df_part['parts_idxs'])[1][2])

In [23]:
df_part.show(10)

+-----------+----------+-----+-----------------+--------------------+-------+----------+--------------------+---+--------------------+--------+--------+-----------+-------------------+---------+----------+--------+--------------------+------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------+--------+--------+--------+--------+--------+--------+
|       Link|      Date|Round|            White|               Black| Result|Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|        Termination|StartTime|   EndDate| EndTime|               Moves|Moves_length|Tournament_long|         Moves_clean|           sf_n1_d22|          sf_n10_d18|            Moves_sf|    parts_idxs|coin_w_d|coin_w_m|coin_w_e|coin_b_d|coin_b_m|coin_b_e|
+-----------+----------+-----+-----------------+--------------------+-------+----------+--------------------+---+--------------------+--------+--------+-----------+

In [24]:
import re

def coin_fraction_deb(coins, parts):
        return float(coins)*2 / 10
    
def coin_fraction_mit(coins, parts):
        return float(coins)*2 / (float(parts[1]) - float(parts[0]))
    
def coin_fraction_end(coins, parts):
        return float(coins)*2 / (float(parts[2]) - float(parts[1]))
    

In [25]:
coin_fraction_d_udf = udf(coin_fraction_deb, FloatType())
coin_fraction_m_udf = udf(coin_fraction_mit, FloatType())
coin_fraction_e_udf = udf(coin_fraction_end, FloatType())

df_part = df_part.withColumn('coin_w_frac_d', coin_fraction_d_udf(df_part['coin_w_d'], df_part['parts_idxs']))
df_part = df_part.withColumn('coin_w_frac_m', coin_fraction_m_udf(df_part['coin_w_m'], df_part['parts_idxs']))
df_part = df_part.withColumn('coin_w_frac_e', coin_fraction_e_udf(df_part['coin_w_e'], df_part['parts_idxs']))

df_part = df_part.withColumn('coin_b_frac_d', coin_fraction_d_udf(df_part['coin_b_d'], df_part['parts_idxs']))
df_part = df_part.withColumn('coin_b_frac_m', coin_fraction_m_udf(df_part['coin_b_m'], df_part['parts_idxs']))
df_part = df_part.withColumn('coin_b_frac_e', coin_fraction_e_udf(df_part['coin_b_e'], df_part['parts_idxs']))

In [26]:
df_part.show(10)

+-----------+----------+-----+---------------+---------------+-------+----------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+
|       Link|      Date|Round|          White|          Black| Result|Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|         Termination|StartTime|   EndDate| EndTime|               Moves|Moves_length|Tournament_long|         Moves_clean|           sf_n1_d22|          sf_n10_d18|            Moves_sf|   parts_idxs|coin_w_d|coin_w_m|coin_w_e|coin_b_d|coin_b_m|coin_b_e|coin_w_frac_d|coin_w_frac_m|coin_w_frac_e|coin_b_frac_d|coin_b_frac_m|coin_b_frac_e|
+---------

### Подсчет доли совпадений для 18 линии

In [27]:
df_part = df_part.withColumn('coin_w_d_18', coincidences_1_udf(df_part['Moves_sf'], df_part['sf_n10_d18'], df_part['parts_idxs'])[0][0])
df_part = df_part.withColumn('coin_w_m_18', coincidences_1_udf(df_part['Moves_sf'], df_part['sf_n10_d18'], df_part['parts_idxs'])[0][1])
df_part = df_part.withColumn('coin_w_e_18', coincidences_1_udf(df_part['Moves_sf'], df_part['sf_n10_d18'], df_part['parts_idxs'])[0][2])

df_part = df_part.withColumn('coin_b_d_18', coincidences_1_udf(df_part['Moves_sf'], df_part['sf_n10_d18'], df_part['parts_idxs'])[1][0])
df_part = df_part.withColumn('coin_b_m_18', coincidences_1_udf(df_part['Moves_sf'], df_part['sf_n10_d18'], df_part['parts_idxs'])[1][1])
df_part = df_part.withColumn('coin_b_e_18', coincidences_1_udf(df_part['Moves_sf'], df_part['sf_n10_d18'], df_part['parts_idxs'])[1][2])

In [28]:
df_part = df_part.withColumn('coin_w_frac_d_18', coin_fraction_d_udf(df_part['coin_w_d_18'], df_part['parts_idxs']))
df_part = df_part.withColumn('coin_w_frac_m_18', coin_fraction_m_udf(df_part['coin_w_m_18'], df_part['parts_idxs']))
df_part = df_part.withColumn('coin_w_frac_e_18', coin_fraction_e_udf(df_part['coin_w_e_18'], df_part['parts_idxs']))

df_part = df_part.withColumn('coin_b_frac_d_18', coin_fraction_d_udf(df_part['coin_b_d_18'], df_part['parts_idxs']))
df_part = df_part.withColumn('coin_b_frac_m_18', coin_fraction_m_udf(df_part['coin_b_m_18'], df_part['parts_idxs']))
df_part = df_part.withColumn('coin_b_frac_e_18', coin_fraction_e_udf(df_part['coin_b_e_18'], df_part['parts_idxs']))

In [29]:
df_part.show(10)

+-----------+----------+-----+---------------+------------------+-------+----------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+
|       Link|      Date|Round|          White|             Black| Result|Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|         Termination|StartTime|   EndDate| EndTime|               Moves|Moves_length|     Tournament_long|         Moves_clean|           sf_n1_d22|          sf_n

### Удаляем ненужное

In [30]:
df_part = df_part.drop(*('Link', 'CurrentPosition', 'EcoUrl', 'Round', 'Termination', 'EndDate','EndTime', 'Tournament_long'))
df_part.show(5)

+----------+-----------------+--------------+------+--------------+---+--------+--------+-----------+---------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+
|      Date|            White|         Black|Result|    Tournament|ECO|WhiteElo|BlackElo|TimeControl|StartTime|               Moves|Moves_length|         Moves_clean|           sf_n1_d22|          sf_n10_d18|            Moves_sf|  parts_idxs|coin_w_d|coin_w_m|coin_w_e|coin_b_d|coin_b_m|coin_b_e|coin_w_frac_d|coin_w_frac_m|coin_w_frac_e|coin_b_frac_d|coin_b_frac_m|coin_b_frac_e|coin_w_d_18|coin_w_m_18|coin_w_e_18|coin_b_d_18|coin_b_m_18|coin_

### Время хода

In [31]:


def motion_times(timeControl, moves, parts):
    all_time_w = timeControl
    inc = 0
    
    if len(all_time_w) > 3:
        inc = float(all_time_w[4])
        all_time_w = float(all_time_w[0:3])    
    else:
        all_time_w = float(all_time_w)
        
    all_time_b = all_time_w
    time_pattern = r'\d+:\d+:\d+\.?\d?'
    times = re.findall(time_pattern, moves)
    times_white_m, times_white_e, times_white_d, times_black_d, times_black_m, times_black_e = [], [], [], [], [], []
    
    for i, time in enumerate(times):
        hours, minutes, seconds = time.split(':')
        now_time = float(hours)*3600 + float(minutes)*60 + float(seconds)
        if i % 2:
            all_time_w += inc
            times_white_d.append(all_time_w - now_time)
            if float(parts[0]) > i:
                times_white_d.append(all_time_w - now_time)
            elif float(parts[1]) > i:
                times_white_m.append(all_time_w - now_time)
            else:
                times_white_e.append(all_time_w - now_time)
            all_time_w = now_time 
        else: 
            all_time_b += inc
            if float(parts[0]) > i:
                times_black_d.append(all_time_w - now_time)
            elif float(parts[1]) > i:
                times_black_m.append(all_time_w - now_time)
            else:
                times_black_e.append(all_time_w - now_time)
            all_time_b = now_time + inc

    return [times_white_d, times_white_m, times_white_e], [times_black_d, times_black_m, times_black_e]

In [32]:
motion_times_udf = udf(motion_times, ArrayType(ArrayType(ArrayType(FloatType()))))

In [33]:
df_part = df_part.withColumn('Moves_time_w', motion_times_udf(df_part['TimeControl'], df_part['Moves'], df_part['parts_idxs'])[1])
df_part = df_part.withColumn('Moves_time_b', motion_times_udf(df_part['TimeControl'], df_part['Moves'], df_part['parts_idxs'])[0])

In [34]:
df_part.show(10)

+----------+-----------+----------------+------+--------------+---+--------+--------+-----------+---------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+
|      Date|      White|           Black|Result|    Tournament|ECO|WhiteElo|BlackElo|TimeControl|StartTime|               Moves|Moves_length|         Moves_clean|           sf_n1_d22|          sf_n10_d18|            Moves_sf|  parts_idxs|coin_w_d|coin_w_m|coin_w_e|coin_b_d|coin_b_m|coin_b_e|coin_w_frac_d|coin_w_frac_m|coin_w_frac_e|coin_b_frac_d|coin_b_frac_m|coin_b_frac_e|coin_w_d_18|coin_w_m_18|coin_w_

### Массив совпадений

In [35]:
def coincidences_array_(moves, sf):

    coincidences_array = []

    for i in range(min(len(moves), len(sf))):
        if moves[i] == sf[i][0]['_1']:
            coincidences_array.append(1)
        else: 
            coincidences_array.append(0)
            
    return coincidences_array

In [36]:
coincidences_array_udf = udf(coincidences_array_, ArrayType(IntegerType()))
df_part = df_part.withColumn('coin_array', coincidences_array_udf(df_part['Moves_sf'], df_part['sf_n1_d22']))
df_part = df_part.withColumn('coin_array_18', coincidences_array_udf(df_part['Moves_sf'], df_part['sf_n10_d18']))

In [37]:
df_part.show(10)

+----------+--------------------+----------------+------+----------+---+--------+--------+-----------+---------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+
|      Date|               White|           Black|Result|Tournament|ECO|WhiteElo|BlackElo|TimeControl|StartTime|               Moves|Moves_length|         Moves_clean|           sf_n1_d22|          sf_n10_d18|            Moves_sf|   parts_idxs|coin_w_d|coin_w_m|coin_w_e|coin_b_d|coin_b_m|coin_b_e|coin_w_frac_d|coin_w_frac_m|coin_w_frac_e|coin_b_frac_d|coin_

### Совпадения в зависимости от времени

In [44]:
def coincidences_time(coin_arr, times_w, times_b, parts):
    
    times_w = times_w[0] + times_w[1] + times_w[2]
    times_b = times_b[0] + times_b[1] + times_b[2]
    
    coincidences_time_w_d, another_time_w_d = [], []
    coincidences_time_b_d, another_time_b_d = [], []
    
    coincidences_time_w_m, another_time_w_m = [], []
    coincidences_time_b_m, another_time_b_m = [], []
    
    coincidences_time_w_e, another_time_w_e = [], []
    coincidences_time_b_e, another_time_b_e = [], []
    
    counter_white, counter_black = 0, 0
    
    for i, res in enumerate(coin_arr):
        if i < float(parts[0]):
            if res:
                if i%2:
                    coincidences_time_w_d.append(float(times_w[counter_white]))
                    counter_white += 1
                else:
                    coincidences_time_b_d.append(float(times_b[counter_black]))
                    counter_black += 1
            else:
                if i%2:
                    another_time_w_d.append(float(times_w[counter_white]))
                    counter_white += 1
                else:
                    another_time_b_d.append(float(times_b[counter_black]))
                    counter_black += 1
        elif i < float(parts[1]):
            if res:
                if i%2:
                    coincidences_time_w_m.append(float(times_w[counter_white]))
                    counter_white += 1
                else:
                    coincidences_time_b_m.append(float(times_b[counter_black]))
                    counter_black += 1
            else:
                if i%2:
                    another_time_w_m.append(float(times_w[counter_white]))
                    counter_white += 1
                else:
                    another_time_b_m.append(float(times_b[counter_black]))
                    counter_black += 1
        else:
            if res:
                if i%2:
                    coincidences_time_w_e.append(float(times_w[counter_white]))
                    counter_white += 1
                else:
                    coincidences_time_b_e.append(float(times_b[counter_black]))
                    counter_black += 1
            else:
                if i%2:
                    another_time_w_e.append(float(times_w[counter_white]))
                    counter_white += 1
                else:
                    another_time_b_e.append(float(times_b[counter_black]))
                    counter_black += 1
            
    
                
    mean_coin_time_w_d = sum(coincidences_time_w_d)/max(len(coincidences_time_w_d), 1)
    mean_another_time_w_d = sum(another_time_w_d)/max(len(another_time_w_d), 1)
    
    mean_coin_time_w_m = sum(coincidences_time_w_m)/max(len(coincidences_time_w_m), 1)
    mean_another_time_w_m = sum(another_time_w_m)/max(len(another_time_w_m), 1)
    
    mean_coin_time_w_e = sum(coincidences_time_w_e)/max(len(coincidences_time_w_e), 1)
    mean_another_time_w_e = sum(another_time_w_e)/max(len(another_time_w_e), 1)
    
    mean_coin_time_b_d = sum(coincidences_time_b_d)/max(len(coincidences_time_b_d), 1)
    mean_another_time_b_d = sum(another_time_b_d)/max(len(another_time_b_d), 1)
    
    mean_coin_time_b_m = sum(coincidences_time_b_m)/max(len(coincidences_time_b_m), 1)
    mean_another_time_b_m = sum(another_time_b_m)/max(len(another_time_b_m), 1)
    
    mean_coin_time_b_e = sum(coincidences_time_b_e)/max(len(coincidences_time_b_e), 1)
    mean_another_time_b_e = sum(another_time_b_e)/max(len(another_time_b_e), 1)

    return mean_coin_time_w_d, mean_another_time_w_d, mean_coin_time_w_m, mean_another_time_w_m, mean_coin_time_w_e,\
        mean_another_time_w_e, mean_coin_time_b_d, mean_another_time_b_d, mean_coin_time_b_m, \
        mean_another_time_b_m, mean_coin_time_b_e, mean_another_time_b_e

In [45]:
coincidences_time_udf = udf(coincidences_time, ArrayType(FloatType()))

df_part = df_part.withColumn('mean_times', coincidences_time_udf(df_part['coin_array'], df_part['Moves_time_w'], df_part['Moves_time_b'], df_part['parts_idxs']))


In [46]:
df_part.show(10)

+----------+-----------------+--------------------+------+--------------+---+--------+--------+-----------+---------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      Date|            White|               Black|Result|    Tournament|ECO|WhiteElo|BlackElo|TimeControl|StartTime|               Moves|Moves_length|         Moves_clean|           sf_n1_d22|          sf_n10_d18|            Moves_sf|   parts_idxs|coin_w_d|coin_w_m|coin_w_e|coin_b_d|coin_b_m|coin_b_e|coin_w_frac_d|coin_w_frac_m|co

### Победа/ ничья

In [ ]:
def spliter(res):
    if res == '1-0':
        return (1,0)
    elif res == '0-1':
        return (0,1)
    else:
        return (0.5, 0.5)

In [ ]:
spliter_udf = udf(spliter, ArrayType(FloatType()))
df_part = df_part.withColumn('Result_w', spliter_udf(df_part['Result'])[0])
df_part = df_part.withColumn('Result_b', spliter_udf(df_part['Result'])[1])

In [ ]:
df_part.show(5)

### Elo рейтинг

In [ ]:
df_players_w = df_part.select(col('White').alias('name'), col('WhiteElo').alias('elo'), col('coin_w_frac').alias('frac'), col('coin_w_frac_18').alias('frac_18'))
df_players_b = df_part.select(col('Black').alias('name'), col('BlackElo').alias('elo'), col('coin_b_frac').alias('frac'), col('coin_b_frac_18').alias('frac_18'))
df_players = df_players_w.union(df_players_b)

In [ ]:
df_players.show(10)

In [ ]:
def sem(arr):
    return np.std(arr, ddof=1)/np.sqrt(len(arr))

def interval():
    

In [ ]:
import numpy as np 
from scipy import stats
bin_size = 50
bins = np.arange(df_players['elo'], df_players['elo'].max() + bin_size, bin_size)
                 
mean_coinc, lower_bounds, upper_bounds = [], [], []

for i in range(len(bins) - 1):
    bin_data = [(df_players['elo'] >= bins[i]) & (df_players['elo'] < bins[i+1])]
    if len(bin_data) > 0:
        frac = np.cocatenate(bin_data['frac'].to_numpy())
        mean = frac.mean()
        std_err = sem(frac)
        ci = stats.t.interval(0.95, len(bin_data) - 1, loc=mean, scale=std_err)
        
        mean_coinc.append(mean)
        lower_bounds.append(ci[0])
        upper_bounds.append(ci[1])
                 
    else:
        mean_coinc.append(np.nan)
        lower_bounds.append(np.nan)
        upper_bounds.append(np.nan)  
                 
bin_centers = bins[:-1] + bin_size / 2

In [ ]:
def searching_bins(elo):
    for i in range(50):
        if ((elo >= lower_bounds[i]) & (elo <= upper_bounds)):
            return lower_bounds[i], bin_centers[i], upper_bound[i]

In [ ]:
searching_bins_udf = udf(searching_bins, ArrayType(StringType()))
df_part = df_part.withColumn('elo_interval_w', searching_bins_udf(df_part['WhiteElo']))
df_part = df_part.withColumn('elo_interval_b', searching_bins_udf(df_part['BlackElo']))

### Те же статистики для прошедших игр

Количество игр за белых у каждого игрока

In [ ]:
grouped = df_part.groupBy(['White'])
grouped.count().show(5)

In [ ]:
window = 100

In [ ]:
df_part_copy = df_part

In [ ]:
df_part = df_part.withColumn('agg_result_w', df_part_copy.groupby('White').mean('Result_w'))
df_part = df_part.withColumn('agg_result_b', df_part_copy.groupby('Black').mean('Result_b'))

### Удаляем ненужное

In [ ]:
df = df.drop(*('Link', 'CurrentPosition', 'EcoUrl', 'Moves', 'Moves_clean', 'sf_n1_d22', 'sf_n10_d18', 'Moves_sf', 'coin_array', 'Data', 'Round', 'Termination', 'StartTime', 'EndDate','EndTime', 'Tournament_long'))

In [ ]:
df.show(10)

In [ ]:
df = df.drop(*('sf_n10_d18', 'Moves_sf'))

In [ ]:
df.show(5)

### Записываю к себе 

In [ ]:
df\
.write\

.mode('overwrite')\
.format('parquet')\
.option('compression', 'gzip')\
.save('hdfs://arnsdpldbr2/user/team/team_uim_crm/MozgolinaAK/chess/data')

In [ ]:
df2 = ss.read.parquet('hdfs://arnsdpldbr2/user/team/team_uim_crm/MozgolinaAK/chess/data')

In [ ]:
df2.show(5)